# Introduce to OpenAI Gym with DuckieNav Environment


We will introduce the main API methods in gym:
* `reset()`
* `step()`
* `render()`

<img style="float: right;" src="images/rl-diagram.png"  width="480" height="480">

and the essentials in RL:
* Environment
* State
* Action
* Reward
* Agent



### Environment
The example is modifed from the Taxi Problem in "Hierarchical Reinforcement Learning with the MAXQ Value Function Decomposition" by  Tom Dietterich (2000), Journal of Artificial Intelligence Research.

<!--img style="float: right;" src="images/DuckieNav-v1.png"  width="240" height="240"-->


```
MAP = [
    "+-----------------+",
    "|O|O| : : : : :G: |",
    "|O|O| |O| |O| |O| |",
    "|O| : |O| |O| |O| |",
    "| : |O|O| : : : : |",
    "| |O|O|O|O|O| |O| |",
    "| : :R: : : : |O| |",
    "| |O|O|O| |O|O|O| |",
    "| |O| : : |O| : : |",
    "| |O| |O|O|O|B|O| |",
    "| : : : : : : |O| |",
    "| |O| |O| |O| |O| |",
    "| : : : : : : |O| |",
    "| |O| |O| |O|O|O| |",
    "| : : :Y: : : : : |",
    "+-----------------+",
]
```

We consider shows a 14 by 9 grid world, except the "service area." The taxi problem is episodic, and in each episode a passenger is located at one of the 4 specially designated locations (R, Y, B, and G). The taxi(agent) starts in a given location and must go to the transported passenger’s location, pick up the passenger, go to the destination location, and put down the passenger. The episode ends when the passenger is deposited at the destination location to one of the 4 locations.


Adapted from https://www.oreilly.com/learning/introduction-to-reinforcement-learning-and-openai-gym


## 1. Initialize DuckieNav-v0

### Installation
You can obtain and install this customized gym environment (https://github.com/ARG-NCTU/gym-duckienav.git): 
```
$ cd ~/gym-duckienav
$ git pull
$ pip install -e . # you may need sudo depending on your setup
```

In [1]:
import gym
import gym_duckienav
import gym.spaces
import numpy as np

env = gym.make("DuckieNav-v0")

### How many 'states' in observation_space: 
There are 2520 states from: 14 (rows) x 9 (columns) x 5 (passenger locations: R, Y, B, G, or on taxi) x 4 (destinations: R, Y, B, or G)

In [2]:
env.observation_space.n

2520

### action_space: 
There are 6 possible actions in Taxi-v2 environment
* down (0), up (1), right (2), left (3), pick-up (4), and drop-off (5)

In [3]:
env.action_space.n

6

## 2. States

Resets the state of the environment and returns an initial observation (state).

The current state is from :
* current taxi row position
* current taxi colum position
* passenger location (Blue or in taxi) from 0: R, 1: G, 2: Y, 3: B; 4: in taxi.
* destination location (Magenta) from 0: R, 1: G, 2: Y, 3: B

In [4]:
env.reset()

print "Current state: " + str(env.s)
for p in env.decode(env.s): print p
env.render()

Current state: 2206
12
2
1
2
+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : :O: |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+



Repeat previous cell for a few times.

In taxi problem, the colors mean:
* blue: passenger's current position
* magenta: destination
* yellow: empty taxi
* green: full taxi

## 3. Actions

Remember that the taxi agent can perform the following actions:
* 0: "South", 
* 1: "North", 
* 2: "East", 
* 3: "West", 
* 4: "Pickup", 
* 5: "Dropoff"

Let's set the state to 124.
Let the taxi agent perform some actions.  

In [5]:
env.s = 124
env.render()

+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : :O: |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+



### `step()`

Run one timestep of the environment's dynamics. 
It returns a tuple (observation, reward, done, info)
* observation (object): agent's observation of the current environment
* reward (float) : amount of reward returned after previous action
* done (boolean): whether the episode has ended, in which case further step() calls will return undefined results
* info (dict): contains auxiliary diagnostic information (helpful for debugging, and sometimes learning)

Essentially the empty taxi is supposed to: 
* move toward the blue letter, 
* pickup the passenger (now the taxi is green), 
* drive to the magenta letter, and 
* drop the passenger (the taxi is yellow again).

It is obvious that we should start with moving "East" env.step(2). Index 2 is for moving "East"
We will do the followings:
* Perform "Pickup" step(4) (although the passenger is not here)
* Perform "East" step(2)
* Perform "Pickup" step(4)
* Perform "West" step(3)
* Perform "South" step(0) for 5 times
* Perfomr "Dropoff" (5)
* Perform "West" step(3) for 4 times
* Perfomr "Dropoff" (5)

In [6]:
state, reward, done, info = env.step(4)
env.render()
print "reward: " + str(reward)

+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : :O: |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (Pickup)
reward: -10


In [7]:
state, reward, done, info = env.step(2)
env.render()
print "reward: " + str(reward)

+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : :O: |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (East)
reward: -1


In [8]:
state, reward, done, info = env.step(4)
env.render()
print "reward: " + str(reward)

+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : :O: |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (Pickup)
reward: -1


In [9]:
state, reward, done, info = env.step(3)
env.render()
print "reward: " + str(reward)

+-----------------+
|O|O| : : : :_:G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : :O: |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (West)
reward: -1


In [10]:
for i in range(0, 5):
    env.step(0)
env.render()

+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : :_:O: |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (South)


In [11]:
state, reward, done, info = env.step(5)
env.render()
print "reward: " + str(reward)

+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : :_:O: |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (Dropoff)
reward: -10


In [12]:
for i in range(0, 4):
    env.step(3)
env.render()

+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : :O: |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (West)


In [13]:
state, reward, done, info = env.step(5)
env.render()
print "reward: " + str(reward)
print "done: " + str(done)

+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : :O: |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (Dropoff)
reward: 20
done: True


### Rewards

You have probably figured out the rewards:
* Perform any movements: -1
* Pick up or drop off at the wrong position: -10
* Drop off the passenger at the right position: 20 

## 4. Ramdon Agent: 

We will use the funciton env.action_space.sample(); you could run the following cell for a few times

In [14]:
print env.action_space.sample()

5


### How good does behaving completely random do?

In [15]:
state = env.reset()

counter = 0
g = 0
reward = None
while reward != 20:
    state, reward, done, info = env.step(env.action_space.sample())
    counter += 1
    g += reward
print("Solved in {} Steps with a total reward of {}".format(counter,g))


Solved in 4021 Steps with a total reward of -15970


You may luck out and solve the environment fairly quickly, but on average, a completely random policy will solve this environment in about ???? steps

## 5. Agent with Basic Reinforcement Learning: Q-Learning

In order to maximize our reward, we will have to have the algorithm remember its actions and their associated rewards. Here, the algorithm’s memory is going to be a Q action value table.

To manage this Q table, we will use a NumPy array. The size of this table will be the number of states (2520) by the number of possible actions (6).

In [16]:
n_states = env.observation_space.n
n_actions = env.action_space.n
Q = np.zeros([n_states, n_actions])

episodes = 1
G = 0
counter = 0
alpha = 0.618

for episode in range(1,episodes+1):
    done = False
    G, reward = 0,0

    state = env.reset()
    env.render()
    
    firstState = state
    print("Initial State = {}".format(state))
    while reward != 20:
        action = np.argmax(Q[state])  #1
        state2, reward, done, info = env.step(action) #2
        Q[state,action] += alpha * (reward + np.max(Q[state2]) - Q[state,action]) #3
        
        if counter < 20:
            print("Step: {}, Action: {}, Reward: {}, Q[{}] \t{}".format(counter, action, reward, state, Q[state]))

        counter += 1
        G += reward
        state = state2
        
finalState = state
print("Final State = {}".format(finalState))
print("Solved in {} Steps with a total reward of {}".format(counter, G))

print Q[finalState]

+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : :O: |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+

Initial State = 924
Step: 0, Action: 0, Reward: -1, Q[924] 	[-0.618  0.     0.     0.     0.     0.   ]
Step: 1, Action: 0, Reward: -1, Q[1104] 	[-0.618  0.     0.     0.     0.     0.   ]
Step: 2, Action: 0, Reward: -1, Q[1284] 	[-0.618  0.     0.     0.     0.     0.   ]
Step: 3, Action: 0, Reward: -1, Q[1464] 	[-0.618  0.     0.     0.     0.     0.   ]
Step: 4, Action: 0, Reward: -1, Q[1644] 	[-0.618  0.     0.     0.     0.     0.   ]
Step: 5, Action: 0, Reward: -1, Q[1824] 	[-0.618  0.     0.     0.     0.     0.   ]
Step: 6, Action: 0, Reward: -1, Q[2004] 	[-0.618  0.     0.     0.     0.     0.   ]
Step: 7, Action: 0, Reward: -1, Q[2184] 	[-0.618  0.     0.     0

First (#1): The agent starts by choosing an action with the highest Q value for the current state using argmax. Argmax will return the index/action with the highest value for that state. Initially, our Q table will be all zeros. But, after every step, the Q values for state-action pairs will be updated.

Second (#2): The agent then takes action and we store the future state as state2 (S t+1). This will allow the agent to compare the previous state to the new state.

Third (#3): We update the state-action pair (St , At) for Q using the reward, and the max Q value for state2 (S t+1). This update is done using the action value formula (based upon the Bellman equation) and allows state-action pairs to be updated in a recursive fashion (based on future values). See the following Figure for the value iteration update.

<img src="images/qlearn.png">

### Let's run over multiple episodes so that we can converge on a optimal policy

In [17]:
episodes = 10000
rewardTracker = []

G = 0
alpha = 0.618

for episode in range(1,episodes+1):
    done = False
    G, reward = 0,0

    state = env.reset()

    while done != True:
        action = np.argmax(Q[state]) 
        state2, reward, done, info = env.step(action) 
        Q[state,action] += alpha * ((reward + (np.max(Q[state2]))  - Q[state,action]))
        G += reward
        state = state2
    
    if episode % 100 == 0:
        print('Episode {} Total Reward: {}'.format(episode,G))
    


Episode 100 Total Reward: -596
Episode 200 Total Reward: -315
Episode 300 Total Reward: -193
Episode 400 Total Reward: 0
Episode 500 Total Reward: -144
Episode 600 Total Reward: -40
Episode 700 Total Reward: -67
Episode 800 Total Reward: -31
Episode 900 Total Reward: -3
Episode 1000 Total Reward: -21
Episode 1100 Total Reward: -612
Episode 1200 Total Reward: -114
Episode 1300 Total Reward: -72
Episode 1400 Total Reward: 5
Episode 1500 Total Reward: 2
Episode 1600 Total Reward: 4
Episode 1700 Total Reward: 6
Episode 1800 Total Reward: 2
Episode 1900 Total Reward: 9
Episode 2000 Total Reward: 4
Episode 2100 Total Reward: 2
Episode 2200 Total Reward: -1
Episode 2300 Total Reward: 4
Episode 2400 Total Reward: 3
Episode 2500 Total Reward: 2
Episode 2600 Total Reward: 4
Episode 2700 Total Reward: 4
Episode 2800 Total Reward: -10
Episode 2900 Total Reward: 2
Episode 3000 Total Reward: 1
Episode 3100 Total Reward: -2
Episode 3200 Total Reward: 7
Episode 3300 Total Reward: 0
Episode 3400 Total 

### Now that we have learned the optimal Q Values we have developed a optimal policy and have no need to train the agent anymore

In [18]:
state = env.reset()

done = None

while done != True:
    # We simply take the action with the highest Q Value
    action = np.argmax(Q[state])
    state, reward, done, info = env.step(action)
    env.render()

+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : :O: |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (North)
+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : :O: |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (North)
+-----------------+
|O|O| : : : : :G: |
|O|O| |O| |O| |O| |
|O| : |O| |O| |O| |
| : |O|O| : : : : |
| |O|O|O|O|O| |O| |
| : :R: : : : :O: |
| |O|O|O| |O|O|O| |
| |O| : : |O| : : |
| |O| |O|O|O|B|O| |
| : : : : : : |O| |
| |O| |O| |O| |O| |
| : : : : : : |O| |
| |O| |O| |O|O|O| |
| : : :Y: : : : : |
+-----------------+
  (North)
+---------